In [1]:
import pandas as pd
import time
from spswarehouse.googleslides import GoogleSlides
from spswarehouse.googlesheets import GoogleSheets

To access your Google Slides, share the file with jupyter-sheets@sps-warehouse.iam.gserviceaccount.com
To access your Google files, share the file with jupyter-sheets@sps-warehouse.iam.gserviceaccount.com


In [ ]:
# The GoogleDrive module of the package doesn't handle copying files right now,
# so we need to build a Drive client directly to handle that.

from oauth2client.service_account import ServiceAccountCredentials
from spswarehouse.credentials import google_config
from googleapiclient.discovery import build

credentials = ServiceAccountCredentials.from_json_keyfile_dict(
    google_config['service-account'],
    scopes=google_config['scopes'],
)

APIGoogleDrive = build(
    'drive',
    'v3',
    credentials=credentials
)

In [ ]:
title_string = "Slide Deck Title - "

high_schools = [
    'High School A',
    'High School B',
]

hs_template_id = '1UJWfeVXMr8dC6sV4Nl4EO0lsKAt1NNuII9V2Di0asrU'

middle_schools = [
    'Middle School A',  
]

ms_template_id = '1OzSRM2vPIlY-EaRpe9UdFj15BDGT1x5_B5JGRc-613M'

In [ ]:
# Open the Data Spreadsheet
data_spreadsheet_id = '1GA3_2WDHGwHb6u-T90eRUueFIRyEFqI3AV0fRzwRfr4'
data_sheet = GoogleSheets.open_by_key(data_spreadsheet_id).sheet1


In [ ]:
# Run this cell if you need to refresh the data
df = pd.DataFrame(data_sheet.get_all_records())

In [ ]:
# The special cases here are related to some particular things with how Summit set up the data

for school in high_schools:
    print(school)
    new_file_response = APIGoogleDrive.files().copy(
        fileId=hs_template_id,
        body={'name': f'{title_string}{school}'},
        supportsTeamDrives=True,
    ).execute()
    print('Template copied')
    new_file_id = new_file_response.get('id')

    this_school = df['School'] == school
    school_df = df[this_school]
    
    # We included multiple years of data for the same data point in our data sheet
    max_year=school_df[["Tag","Academic Year"]].groupby(["Tag"]).max()

    # Insert the school name into the deck
    GoogleSlides.presentations().batchUpdate(
        body={
            "requests": [
                {
                    "replaceAllText": {
                        "containsText": {
                            "text": '{school}'
                        },
                        "replaceText": school,
                        "pageObjectIds": [],
                    }
                }
            ]
        },
        presentationId=new_file_id
    ).execute()

    # Iterate through all the data points for the school
    for index, row in school_df.iterrows():
        # Only report the most recent year for each data point
        if row["Academic Year"] != max_year.loc[row['Tag']][0]:
            continue
        
        # The "Goal" rows only have a percentage goal
        if row['Category']=='Goal':
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
        
        
        # "Enrollment" is the only data point that is a raw number, and not a percentage
        elif row['Excellence Goal'] == 'Enrollment':
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_year}}'
                                },
                                "replaceText": str(row["Academic Year"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
        
        # All other data points have a percentage + the raw numbers
        else:
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}%}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()

            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_num}}'
                                },
                                "replaceText": str(row["Numerator"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()

            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_den}}'
                                },
                                "replaceText": str(row["Denominator"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_year}}'
                                },
                                "replaceText": str(row["Academic Year"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
        
        
        # This line is very important - it prevents the notebook from overrunning the Slides API quota
        time.sleep(4)
        
        
# By default, the file should inherit the folder permissions, but if you need to change that:
#     from spswarehouse.googledrive import GoogleDrive # There's probably a way to use the API for this, but I'm lazy.
#     GoogleDrive.CreateFile({'id': new_file_id}).InsertPermission({'value': '<e-mail>', 'role': 'writer', 'type': 'user'})

In [ ]:
# Identical to above, except all references to high school changed to middle school

for school in middle_schools:
    print(school)
    new_file_response = APIGoogleDrive.files().copy(
        fileId=ms_template_id,
        body={'name': f'{title_string}{school}'},
        supportsTeamDrives=True,
    ).execute()
    print('Template copied')
    new_file_id = new_file_response.get('id')

    this_school = eg_df['School'] == school
    school_df = eg_df[this_school]
    max_year=school_df[["Tag","Academic Year"]].groupby(["Tag"]).max()

    GoogleSlides.presentations().batchUpdate(
        body={
            "requests": [
                {
                    "replaceAllText": {
                        "containsText": {
                            "text": '{school}'
                        },
                        "replaceText": school,
                        "pageObjectIds": [],
                    }
                }
            ]
        },
        presentationId=new_file_id
    ).execute()

    for index, row in school_df.iterrows():
        if row["Academic Year"] != max_year.loc[row['Tag']][0]:
            continue
            
        if row['Category']=='Goal':
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            
        elif row['Excellence Goal'] == 'Enrollment':
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_year}}'
                                },
                                "replaceText": str(row["Academic Year"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            
        else:
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}%}}'
                                },
                                "replaceText": str(row["Main Data"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()

            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_num}}'
                                },
                                "replaceText": str(row["Numerator"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()

            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_den}}'
                                },
                                "replaceText": str(row["Denominator"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            
            GoogleSlides.presentations().batchUpdate(
                body={
                    "requests": [
                        {
                            "replaceAllText": {
                                "containsText": {
                                    "text": f'{{{row["Tag"]}_year}}'
                                },
                                "replaceText": str(row["Academic Year"]),
                                "pageObjectIds": [],
                            }
                        }
                    ]
                },
                presentationId=new_file_id
            ).execute()
            
        time.sleep(4)

# Additional code: Deleting a File

In case you need to delete a file that you created

In [ ]:
# List all the files in your Drive
from spswarehouse.googledrive import GoogleDrive
fileList = GoogleDrive.ListFile({'q': "trashed = False"}).GetList()
for file in fileList:
    print('Title: %s, ID: %s' % (file['title'], file['id']))

In [ ]:
# Copy and paste the "ID" of the file you need to delete into this code
GoogleDrive.CreateFile({'id': 'id of drive file'}).Delete()